In [ ]:
import os
import pandas as pd
import SimpleITK as sitk
from radiomics import featureextractor
import numpy as np
from tqdm import tqdm

In [ ]:

# ===== Path Configuration =====
converted_dir = r"D:\research\cd-ml\H\converted_nii"
roi_root = r"D:\research\cd-ml\H"
output_excel = os.path.join(converted_dir, "radiomics_features.xlsx")
failed_csv = os.path.join(converted_dir, "failed_cases.csv")

In [ ]:

# ===== Initialize PyRadiomics Extractor =====
extractor = featureextractor.RadiomicsFeatureExtractor()



In [ ]:
# ===== Containers for Results =====
ct_results = []
pet_results = []
failed_cases = []


In [ ]:

# ===== Utility Functions =====
def load_nii(path):
    try:
        img = sitk.ReadImage(path)
        print(f"🟢 Loaded with SimpleITK: {path}")
        return img
    except Exception:
        print(f"⚠ SimpleITK failed, fallback to nibabel: {path}")
        import nibabel as nib
        nib_img = nib.load(path)
        data = nib_img.get_fdata().astype("float32")
        sitk_img = sitk.GetImageFromArray(data)
        spacing = tuple(map(float, nib_img.header.get_zooms()[:3]))
        sitk_img.SetSpacing(spacing)
        sitk_img.SetOrigin((0.0, 0.0, 0.0))
        print(f"✅ Converted nibabel to SimpleITK: {path}")
        return sitk_img

def resample_to_target(image, reference):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(reference)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(0)
    return resampler.Execute(image)

def find_matching_folder(parent_dir, target_name):
    for folder in os.listdir(parent_dir):
        if folder.endswith("_CT") and target_name in folder:
            return os.path.join(parent_dir, folder)
        if folder.endswith("_PET") and target_name in folder:
            return os.path.join(parent_dir, folder)
    return None

def find_first_nii(root_path):
    for root, _, files in os.walk(root_path):
        for f in files:
            if f.endswith(".nii") or f.endswith(".nii.gz"):
                return os.path.join(root, f)
    return None


In [ ]:

# ===== Main Extraction Loop =====
for foldername in os.listdir(roi_root):
    if "ROI" not in foldername:
        continue

    roi_folder = os.path.join(roi_root, foldername)
    for patient_name in os.listdir(roi_folder):
        patient_path = os.path.join(roi_folder, patient_name)
        if not os.path.isdir(patient_path):
            continue

        mask_path = find_first_nii(patient_path)
        if not mask_path:
            failed_cases.append({"patient": patient_name, "reason": "No PET mask found"})
            continue

        ct_dir = find_matching_folder(converted_dir, patient_name)
        pet_dir = find_matching_folder(converted_dir, patient_name.replace("_CT", "_PET"))
        if ct_dir is None or pet_dir is None:
            failed_cases.append({"patient": patient_name, "reason": "CT or PET image folder not found"})
            continue

        try:
            ct_img_path = next(os.path.join(ct_dir, f) for f in os.listdir(ct_dir) if f.endswith(".nii") or f.endswith(".nii.gz"))
            pet_img_path = next(os.path.join(pet_dir, f) for f in os.listdir(pet_dir) if f.endswith(".nii") or f.endswith(".nii.gz"))
        except:
            failed_cases.append({"patient": patient_name, "reason": "CT or PET image file not found"})
            continue

        try:
            ct_img = load_nii(ct_img_path)
            pet_img = load_nii(pet_img_path)
            mask = load_nii(mask_path)

            mask_array = sitk.GetArrayViewFromImage(mask)
            labels = np.unique(mask_array)
            labels = labels[labels != 0]
            if len(labels) == 0:
                raise ValueError("No foreground labels found in mask")
            selected_label = int(labels.max())

            if ct_img.GetSize() != mask.GetSize():
                ct_img = resample_to_target(ct_img, mask)
            if pet_img.GetSize() != mask.GetSize():
                pet_img = resample_to_target(pet_img, mask)

            ct_features = extractor.execute(ct_img, mask, label=selected_label)
            ct_features["patient"] = patient_name
            ct_results.append(ct_features)

            pet_features = extractor.execute(pet_img, mask, label=selected_label)
            pet_features["patient"] = patient_name
            pet_results.append(pet_features)

            print(f"✅ Features extracted for {patient_name}")

        except Exception as e:
            failed_cases.append({"patient": patient_name, "reason": str(e)})
            print(f"❌ Failed to process {patient_name}: {e}")
            continue



In [ ]:

# ===== Save Results =====
ct_df = pd.DataFrame(ct_results)
pet_df = pd.DataFrame(pet_results)

with pd.ExcelWriter(output_excel) as writer:
    ct_df.to_excel(writer, sheet_name="CT", index=False)
    pet_df.to_excel(writer, sheet_name="PET", index=False)

pd.DataFrame(failed_cases).to_csv(failed_csv, index=False)

print("\n🎉 Radiomics feature extraction completed.")
print(f"📄 Results saved to: {output_excel}")
print(f"📄 Failed cases saved to: {failed_csv}")